In [11]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

In [3]:
ds = load_dataset("searle-j/kote", trust_remote_code=True)
print(ds)

Generating train split: 40000 examples [00:00, 115071.17 examples/s]
Generating test split: 5000 examples [00:00, 115001.29 examples/s]
Generating validation split: 5000 examples [00:00, 115012.64 examples/s]

DatasetDict({
    train: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 40000
    })
    test: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['ID', 'text', 'labels'],
        num_rows: 5000
    })
})


In [4]:
df_train = pd.DataFrame(ds['train'])
df_train

,ID,text,labels
0,39087,내가 톰행크스를 좋아하긴 했나보다... 초기 영화 빼고는 다 봤네.,"[2, 13, 15, 16, 29, 39]"
1,30893,"정말 상상을 초월하는 무개념 진상들 상대하다 우울증, 공항장애 걸리는 공무원 많아요...","[0, 5, 7, 10, 19, 22, 29, 35, 36, 38]"
2,45278,"새로운 세상과 조우한 자의 어린아이 같은 반응, 어쩌면 회복된 것은 눈이 아닌 순수...","[1, 2, 7]"
3,16398,미역은 원생생물계 산호초는 동물ㅇㅇ 아 미역이 바다의 새ㄱㅇㄱㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,"[9, 15, 20, 23, 26, 28, 29]"
4,13653,네 맞습니다 플스는 역시 30프레임이 어울리죠 ㅎ,"[1, 2, 8, 9, 11, 13, 15, 16, 28, 29, 32, 40, 42]"
...,...,...,...
39995,11284,멋지게 잘나왔네요! 고생하신보람이있겠습니다.,"[1, 2, 4, 7, 8, 13, 15, 16, 28, 29, 32, 39, 40..."
39996,44732,고막주의라 그래 잔뜩 긴장햇는데 에이든은 우는것도 귀엽네영,"[15, 16, 32, 41]"
39997,38158,박정호 교수님은 sbs 블루베리부터 왕팬입니다.,"[1, 7, 8, 13, 16, 24, 42]"
39998,860,겁나 어메이징한 영화! 에이미란 이름엔 저주가 얹은듯,"[2, 9, 10, 15, 18, 23, 33, 35, 39]"


In [6]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("CUDA version:", torch.version.cuda)
print("Device count:", torch.cuda.device_count())
if torch.cuda.is_available():
    print("Current GPU:", torch.cuda.get_device_name(0))

PyTorch version: 2.5.1+cu124
CUDA available: True
CUDA version: 12.4
Device count: 2
Current GPU: NVIDIA GeForce RTX 4090


In [7]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# PyTorch에서 GPU 설정 확인
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# Hugging Face에서 토크나이저와 모델 불러오기
tokenizer = AutoTokenizer.from_pretrained("searle-j/kote_for_easygoing_people", trust_remote_code=True)
model = AutoModelForSequenceClassification.from_pretrained("searle-j/kote_for_easygoing_people")

# 감정 분석 모델 설정 
pipe = TextClassificationPipeline(
    model=model.to(device),  # 모델을 GPU로 이동
    tokenizer=tokenizer,
    device=0 if device == "cuda" else -1,  # GPU(0) 또는 CPU(-1)
    return_all_scores=True,
    function_to_apply='sigmoid'
)

Using device: cuda


Device set to use cuda:0


In [8]:
finish = pd.read_csv('best_comments_with_episodes.csv')
webtoon = pd.read_csv('webtoon_best_comments_final.csv')

In [12]:
def perform_sentiment_analysis(text_list):
    """
    감정 분석을 수행하여 각 텍스트에 대한 가장 높은 점수의 레이블을 반환합니다.

    Parameters:
        text_list (list): 텍스트 데이터 리스트

    Returns:
        list: 각 텍스트에 대한 감정 분석 결과 레이블
    """
    emotions = []
    for text in tqdm(text_list, desc="Performing Sentiment Analysis", unit="comment"):
        pipe_text = pipe([text])  # pipe는 개별적으로 호출
        highest_label = max(pipe_text[0], key=lambda x: x["score"])["label"]
        emotions.append(highest_label)
    return emotions


def save_with_emotions(df, text_column, output_file):
    """
    데이터프레임에서 댓글 열을 분석하고 감정 결과를 추가한 후 저장합니다.

    Parameters:
        df (pd.DataFrame): 감정 분석을 수행할 데이터프레임
        text_column (str): 댓글이 있는 열 이름
        output_file (str): 저장할 파일 이름
    """
    text_list = df[text_column].tolist()
    print(f"Processing {output_file}...")
    
    # 감정 분석 수행
    df["emotion"] = perform_sentiment_analysis(text_list)
    
    # 결과 저장
    df.to_csv(output_file, index=False, encoding="utf-8-sig")
    print(f"{output_file} 저장 완료!")

In [13]:
# 연재중 웹툰 댓글 감정 분석
save_with_emotions(webtoon, text_column="Comment", output_file="webtoon_emotion.csv")

# 완결결 웹툰 댓글 감정 분석
save_with_emotions(finish, text_column="Comment", output_file="finish_emotion.csv")

Processing webtoon_emotion.csv...


Performing Sentiment Analysis: 100%|██████████| 37574/37574 [01:23<00:00, 450.98comment/s]


webtoon_emotion.csv 저장 완료!
Processing finish_emotion.csv...


Performing Sentiment Analysis: 100%|██████████| 46549/46549 [01:42<00:00, 452.67comment/s]


finish_emotion.csv 저장 완료!
